In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
torch.cuda.empty_cache() 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
import os
# os.chdir("/content/xinyu_exp/Colab Notebooks/xinyu_result/") 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# os.chdir("/MyDrive/Colab Notebooks/ori_data")
path = "/content/drive/MyDrive"
dirs = os.listdir(path)
for file in dirs:
   print(file)

Colab Notebooks
longformer-baseline-tim.ipynb
Longformer-baseline-YC.ipynb
Model
.ipynb_checkpoints
BiggerlLSTM.ipynb
xinyu_result
BERT_experiment_XinyuZhao.ipynb


In [ ]:
import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers
# from tqdm import tqdm
import pandas as pd
# from tensorflow import keras

# from tensorflow.data import Dataset
DIR = "/content/drive/MyDrive/xinyu_result/"
TRAIN_CSV = "/content/drive/MyDrive/Colab Notebooks/ori_data/train.csv"
FILENAME_DIR = "/content/drive/MyDrive/Colab Notebooks/ori_data/subset_name.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/xinyu_result/"
!pip install -q transformers
!pip install -q datasets
from datasets import Dataset, load_metric
import nltk
nltk.download("popular")

     |████████████████████████████████| 3.8 MB 15.3 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 58.6 MB/s 
     |████████████████████████████████| 895 kB 68.3 MB/s 
     |████████████████████████████████| 312 kB 14.0 MB/s 
     |████████████████████████████████| 1.1 MB 90.3 MB/s 
     |████████████████████████████████| 134 kB 96.1 MB/s 
     |████████████████████████████████| 212 kB 98.9 MB/s 
     |████████████████████████████████| 127 kB 95.5 MB/s 
     |████████████████████████████████| 144 kB 89.7 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 96.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

True

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizer
from transformers import DataCollatorWithPadding

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
fileIDs = pd.read_csv(FILENAME_DIR)
fileIDs = fileIDs.values.tolist()

In [ ]:
MAX_LENGTH = 512
label_dict = {'Lead': 0, 'Position': 1, 'Evidence': 2, 'Claim': 3, 'Concluding Statement': 4,
             'Counterclaim': 5, 'Rebuttal': 6}
train = pd.read_csv(TRAIN_CSV, header=0)
text = []
label = []
label_text = []
max_len = 0
n = len(fileIDs)
count = 0
for i in range(n):
  ID = fileIDs[i][0]
  data = train.loc[train['id'] == ID]
  temp_text = data.loc[:, 'discourse_text']
  temp_labels = data.loc[:, 'discourse_type']
  for i in range(len(data)):
    single_text = temp_text.iloc[i]
    single_text = nltk.tokenize.sent_tokenize(single_text)
    single_label_text = temp_labels.iloc[i]
    single_label = label_dict[single_label_text]
    for line in single_text:
      if len(line) >= 3 and count <= 6000:
        max_len = max(max_len, len(line))
        line = line.replace("\r", " ")
        line = line.replace("\n", " ")
        line = line.replace("\xa0", " ")
        line = line.replace("'", "")
        line = line.replace('"', "")
        line = line.strip()
        line = str(line)
        text.append(line)
        label_text.append(single_label_text)
        label.append(int(single_label))
        count += 1
print(max_len)

627


In [ ]:
data = pd.DataFrame([text, label, label_text])
data = data.transpose()
data.columns = ['text', 'label', 'label_text']
data.head()
train_data = data.sample(frac=0.8, random_state=0, axis=0)
val_data = data[~data.index.isin(train_data.index)]
print(len(train_data), len(val_data))
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
val_dataset = val_dataset.remove_columns(['__index_level_0__'])

4801 1200


In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 4801
})

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding='max_length', max_length=MAX_LENGTH, truncation=True)
# def preprocess_function(examples):
#     return tokenizer(examples["text"], padding=True, truncation=True)

In [ ]:
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_val_dataset = val_dataset.map(preprocess_function, batched=True)
encoded_train_dataset

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4801
})

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=7)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    no_cuda=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.190699,0.598333
2,1.283900,1.148757,0.623333
3,1.283900,1.109484,0.634167
4,0.985900,1.137534,0.645000
5,0.764800,1.173732,0.620833
6,0.764800,1.216821,0.624167
7,0.586100,1.251009,0.621667
8,0.586100,1.268194,0.620833


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, label_text. If text, label_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/xinyu_result/checkpoint-301
Configuration saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/checkpoint-301/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/checkpoint-301/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/checkpoint-301/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/checkpoint-301/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceCl

TrainOutput(global_step=2408, training_loss=0.8382919919847254, metrics={'train_runtime': 2360.0729, 'train_samples_per_second': 16.274, 'train_steps_per_second': 1.02, 'total_flos': 1.01060230834176e+16, 'train_loss': 0.8382919919847254, 'epoch': 8.0})

In [ ]:
trainer.evaluate()
# trainer.save_model(OUTPUT_DIR)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, label_text. If text, label_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 16


{'epoch': 8.0,
 'eval_accuracy': 0.645,
 'eval_loss': 1.1375341415405273,
 'eval_runtime': 23.5598,
 'eval_samples_per_second': 50.934,
 'eval_steps_per_second': 3.183}

In [ ]:
trainer.save_model(OUTPUT_DIR)

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/xinyu_result/
Configuration saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/xinyu_result/special_tokens_map.json


###TEST###

In [ ]:
from transformers import BertTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/xinyu_result/")
model.to(device)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# DIR = "/content/drive/MyDrive/Colab Notebooks/ori_data/"
label_dict = {'Lead': 0, 'Position': 1, 'Evidence': 2, 'Claim': 3, 'Concluding Statement': 4,
            'Counterclaim': 5, 'Rebuttal': 6}
fileIDs = pd.read_csv(DIR+'val.csv')
fileIDs = fileIDs.values.tolist()
fileIDs = fileIDs[0:40]
print(fileIDs)
train = pd.read_csv(DIR+'train.csv', header=0)
text = []
label = []
label_text = []
max_len = 0
n = len(fileIDs)
print(n)
count = 0
for i in range(n):
  ID = fileIDs[i][0]
  data = train.loc[train['id'] == ID]
  temp_text = data.loc[:, 'discourse_text']
  temp_labels = data.loc[:, 'discourse_type']
  for i in range(len(data)):
    single_text = temp_text.iloc[i]
    single_text = nltk.tokenize.sent_tokenize(single_text)
    single_label_text = temp_labels.iloc[i]
    single_label = label_dict[single_label_text]
    for line in single_text:
      if len(line) >= 3 and count <= 100:
        max_len = max(max_len, len(line))
        line = line.replace("\r", " ")
        line = line.replace("\n", " ")
        line = line.replace("\xa0", " ")
        line = line.replace("'", "")
        line = line.replace('"', "")
        line = line.strip()
        line = str(line)
        text.append(line)
        label_text.append(single_label_text)
        label.append(int(single_label))
        count += 1

[['F9368F6BAB62'], ['D4AA8EC389C6'], ['FC9E2A38AFBB'], ['01495BF80A2A'], ['2DEC8D3BA891'], ['0A2EE9B6BF75'], ['412227C301CD'], ['D98F3E1DE828'], ['AD7012F5871A'], ['32A121824D02'], ['09EE9A3D3248'], ['C42EBA5B3529'], ['29067E722A0A'], ['89E1BF0D40C8'], ['B3C82409B8C2'], ['26370DAD607F'], ['81BCB1DE23B7'], ['185FD725317F'], ['027616F7ED1D'], ['6068A54ED99D'], ['BF040431FB95'], ['BF97DBDFC416'], ['7F74864D0865'], ['39CAEB5135DB'], ['FF3F4514BF85'], ['1F8DDCFCA766'], ['951A053453C0'], ['92933212F4D9'], ['B78A191477C9'], ['C440D9C8CE92'], ['EB94208AC8AF'], ['4933B56F9E7B'], ['FBF6DE55FBB7'], ['68E95519CBCA'], ['D930E883CCBC'], ['2B002FEE337E'], ['3FD462C874BF'], ['CFCB9135F1BF'], ['CAC42EC32F44'], ['0CF61A389E37']]
40


In [ ]:
!nvidia-smi

Wed Mar  9 21:12:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    34W / 250W |   1469MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# val_text, val_label, val_label_text = make_dataset(DIR+'val.csv')
tokenized_texts = tokenizer(text, padding='max_length', max_length=128, truncation=True)
inputs = {k: torch.tensor(v).to(device) for k, v in tokenized_texts.items()}

In [ ]:
output = model(**inputs)
predict_result = torch.argmax(output.logits.cpu(), axis=1)
final_result = metric.compute(predictions=predict_result, references=label)
print(f"{output} \npredicted_labels: {predict_result} \nFinal Accuracy on inference dataset:{final_result}")

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.9841e-01,  1.9111e+00, -9.1453e-01, -6.7349e-01,  1.4452e+00,
         -1.3200e+00, -1.3711e+00],
        [-7.6253e-02, -2.6144e+00,  4.2819e+00,  8.4581e-02,  9.6988e-02,
         -2.1965e+00, -1.5087e+00],
        [ 1.0847e+00, -1.5061e+00,  2.2710e+00, -4.2467e-01, -9.2929e-01,
         -1.6200e+00, -1.8506e+00],
        [ 2.6874e-01, -2.1315e+00,  4.0501e+00, -4.3536e-01, -6.3074e-01,
         -2.0363e+00, -1.2181e+00],
        [ 1.2749e+00, -1.7437e+00,  2.0741e+00, -5.9656e-01, -9.3048e-01,
         -1.7119e+00, -1.7941e+00],
        [-6.6358e-01, -2.2941e+00,  3.1457e+00,  8.6770e-01, -3.5195e-01,
         -1.6457e+00, -1.3346e+00],
        [ 3.0533e-01, -2.0146e+00,  2.6525e+00,  4.1644e-01, -5.9863e-01,
         -2.1584e+00, -1.6868e+00],
        [ 1.0251e+00, -1.4553e+00,  2.2937e+00, -7.4946e-01, -5.9531e-01,
         -1.7932e+00, -1.3937e+00],
        [ 6.7407e-01, -1.9857e+00,  2.4744e+00, -3.9861e-01, -8.4712e-01,
   

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
label_dict = {'Lead': 0, 'Position': 1, 'Evidence': 2, 'Claim': 3, 'Concluding Statement': 4,
            'Counterclaim': 5, 'Rebuttal': 6}
print("\n")
label_text = ["L", "P", "E", "Claim", "Con", "Counter", "Rebuttal"]
print(classification_report(label, predict_result.tolist(), 
                      target_names=["Lead", "Position", "Evidence", "Claim", "Concluding Statement", "Counterclaim", "Rebuttal"]))
cfmx = confusion_matrix(predict_result, label)
cfmx = pd.DataFrame(cfmx, index = ['true:' + name for name in label_text], columns=[name for name in label_text])
print("\n")
print(cfmx)



                      precision    recall  f1-score   support

                Lead       1.00      0.67      0.80         3
            Position       1.00      0.83      0.91         6
            Evidence       0.85      1.00      0.92        62
               Claim       0.88      0.47      0.61        15
Concluding Statement       0.67      0.75      0.71         8
        Counterclaim       0.75      0.60      0.67         5
            Rebuttal       0.00      0.00      0.00         2

            accuracy                           0.84       101
           macro avg       0.73      0.62      0.66       101
        weighted avg       0.83      0.84      0.82       101



               L  P   E  Claim  Con  Counter  Rebuttal
true:L         2  0   0      0    0        0         0
true:P         0  5   0      0    0        0         0
true:E         1  0  62      6    2        1         1
true:Claim     0  0   0      7    0        0         1
true:Con       0  1   0      1    6 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
